## Requirements
1. 600 to 700k (put 800k to get a better overall picture)

2. Has to be 4 bedroom

3. Closer to central (closer to central will take the priority) - it has to have established MRT allowing user to travel to central areas

4. Remaining lease has to be 70 years at least - the house has been use for 20 years 

5. Other considerations: will want to move away by end of 5 years

6. avoid being too close to main roads or MRT, directly beside mrt as there will be too much noise

 - 3 bedroom flat - to show in terms of square meters is bigger than flat 4 room flat

7. filter any good houses that fit the criteria and extract the finished results on to a csv allowing user to decide what areas are good.

8. also create an overview of 4 bedroom flats in singapore which account for their price and towns

In [5]:
# ------------------------------------------------------------
# 0️⃣  IMPORTS
# ------------------------------------------------------------
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import folium
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')


In [9]:
# ------------------------------------------------------------
# 1️⃣  DEFINE THE DIRECTORY THAT CONTAINS THE FOUR FILES
# ------------------------------------------------------------
base_dir = Path(r"C:\Users\Admin\HDB")      # <-- change only this line if you move the folder

csv_path            = base_dir / "resale_flat_prices.csv"
mrt_path            = base_dir / "LTAMRTStationExitGEOJSON.geojson"
roads_kml_path      = base_dir / "RoadNetwork.kml"
blocks_geojson_path = base_dir / "HDB_Block_Polygons.geojson"


In [10]:
# ------------------------------------------------------------
# 1️⃣  LOAD RESALE CSV
# ------------------------------------------------------------
resale = pd.read_csv(csv_path)
print(f"Resale rows loaded: {resale.shape[0]:,}")

# ------------------------------------------------------------
# 2️⃣  LOAD MRT EXITS (GeoJSON)
# ------------------------------------------------------------
mrt = gpd.read_file(mrt_path)
print(f"MRT exits loaded: {mrt.shape[0]:,}")

# ------------------------------------------------------------
# 3️⃣  LOAD ROAD NETWORK (KML) – try direct read first
# ------------------------------------------------------------
try:
    roads = gpd.read_file(roads_kml_path, driver='KML')
    print(f"Road network loaded directly from KML: {roads.shape[0]:,}")
except Exception as e:
    # fallback: convert to GeoJSON once using ogr2ogr (GDAL)
    print("Direct KML read failed – converting to GeoJSON (one‑time operation).")
    import subprocess, sys
    tmp_geojson = base_dir / "RoadNetwork_converted.geojson"
    subprocess.check_call(["ogr2ogr", "-f", "GeoJSON", str(tmp_geojson), str(roads_kml_path)])
    roads = gpd.read_file(tmp_geojson)
    print(f"Road network loaded after conversion: {roads.shape[0]:,}")

# ------------------------------------------------------------
# 4️⃣  LOAD HDB BLOCK POLYGONS (GeoJSON)
# ------------------------------------------------------------
blocks = gpd.read_file(blocks_geojson_path)
print(f"HDB block polygons loaded: {blocks.shape[0]:,}")


Resale rows loaded: 215,121
MRT exits loaded: 563
Road network loaded directly from KML: 61,822
HDB block polygons loaded: 12,847


## latest ai response follow this as guideline

In [11]:
# ------------------------------------------------------------
#  COMPLETE HDB ANALYSIS (single‑cell version – fixed)
# ------------------------------------------------------------
import re, warnings, numpy as np, pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
warnings.filterwarnings('ignore')

print("="*70)
print("🏠 COMPLETE HDB ANALYSIS WITH POLYGON DATA")
print("="*70)

# ------------------------------------------------------------
# 1️⃣  PREPARE DATAFRAMES
# ------------------------------------------------------------
# (Assumes `resale` and `blocks` are already loaded)
for df in (resale, blocks):
    df.columns = df.columns.str.lower().str.strip()

print("\nColumns after normalising")
print("Resale :", list(resale.columns))
print("Blocks :", list(blocks.columns))

# ------------------------------------------------------------
# 2️⃣  EXTRACT BLOCK / STREET / TOWN FROM BLOCK DESCRIPTIONS
# ------------------------------------------------------------
def extract_block_info_from_description(description: str) -> dict:
    """Parse HTML description and return block, street and town (or None)."""
    try:
        soup = BeautifulSoup(description, "html.parser")
        txt  = soup.get_text(separator=" ").upper()          # work in upper‑case

        # ----- block & street -----
        blk = re.search(r'\b(?:BLK|BLOCK)\s*(\d+[A-Z]?)\s+([^,\n]+)', txt)
        if blk:
            block, street = blk.group(1).strip(), blk.group(2).strip()
        else:
            block_match = re.search(r'\b\d{1,4}[A-Z]?\b', txt)
            block = block_match.group(0).strip() if block_match else None
            street = None

        # ----- town (last line) -----
        lines = [ln.strip() for ln in txt.split("\n") if ln.strip()]
        town = lines[-1] if lines and len(lines[-1]) > 3 and not lines[-1].isdigit() else None

        return {"block": block, "street_name": street, "town": town}
    except Exception:
        return {"block": None, "street_name": None, "town": None}

# ---- quick test on first 3 rows ---------------------------------
print("\n🔍 TESTING DESCRIPTION EXTRACTION (first 3 rows)")
for i in range(min(3, len(blocks))):
    print(f"\nRow {i} – description snippet:")
    print(blocks.iloc[i]["description"][:120] + "...")
    print("Extracted →", extract_block_info_from_description(blocks.iloc[i]["description"]))

# ---- run on EVERY block -----------------------------------------
print("\n⚙️  EXTRACTING BLOCK INFO FROM ALL DESCRIPTIONS …")
extracted = [extract_block_info_from_description(d) for d in blocks["description"]]
extracted_df = pd.DataFrame(extracted)

# ---- concatenate AND REMOVE DUPLICATE column names ---------------
# keep the **last** occurrence (the newly‑extracted values)
blocks = pd.concat([blocks.reset_index(drop=True), extracted_df], axis=1)
blocks = blocks.loc[:, ~blocks.columns.duplicated(keep='last')]   # <<< FIX !!!

print("\n✅ Extraction completed!")
print(f"Blocks with block number : {blocks['block'].count():,}")
print(f"Blocks with street name  : {blocks['street_name'].count():,}")
print(f"Blocks with town         : {blocks['town'].count():,}")

# ------------------------------------------------------------
# 3️⃣  CLEAN & NORMALISE JOIN KEYS
# ------------------------------------------------------------
required_cols = ["block", "street_name", "town"]
for col in required_cols:
    # resale side (must exist)
    if col in resale.columns:
        resale[col] = resale[col].astype(str).str.strip().str.upper()
    else:
        raise KeyError(f"Column '{col}' missing from resale data")

    # blocks side – already upper‑cased above
    if col in blocks.columns:
        blocks[col] = blocks[col].astype(str).str.strip().str.upper()
        blocks.loc[blocks[col] == "NONE", col] = None

print("\n🔧 AFTER CLEANING")
print(f"Resale shape : {resale.shape}")
print(f"Blocks shape : {blocks.shape}")

def drop_missing(df, name):
    before = len(df)
    df = df.dropna(subset=required_cols)
    for c in required_cols:
        df = df[~df[c].isin(["", "NONE", "NAN"])]
    print(f"{name}: removed {before - len(df):,} rows lacking join keys")
    return df

resale = drop_missing(resale, "Resale")
blocks = drop_missing(blocks, "Blocks")

# ------------------------------------------------------------
# 4️⃣  JOIN‑KEY DIAGNOSTIC (optional)
# ------------------------------------------------------------
resale["join_key"] = resale["block"] + "|" + resale["street_name"] + "|" + resale["town"]
blocks["join_key"] = blocks["block"] + "|" + blocks["street_name"] + "|" + blocks["town"]

res_keys   = set(resale["join_key"])
blk_keys   = set(blocks["join_key"])
matched    = res_keys.intersection(blk_keys)

print("\n🔗 JOIN‑KEY ANALYSIS")
print(f"Resale keys   : {len(res_keys):,}")
print(f"Block keys    : {len(blk_keys):,}")
print(f"Matched keys  : {len(matched):,}")
print(f"Match rate    : {len(matched) / len(res_keys) * 100:.1f}%")

# clean temporary keys
resale = resale.drop(columns=["join_key"])
blocks = blocks.drop(columns=["join_key"])

# ------------------------------------------------------------
# 5️⃣  PREPARE RESALE DATA (lease, age, price‑per‑sqm)
# ------------------------------------------------------------
def lease_str_to_years(txt):
    """Convert strings like '73 years 6 months' → float years."""
    if pd.isna(txt):
        return np.nan
    txt = str(txt).lower()
    yrs, mos = 0, 0
    if "year" in txt:
        try:
            yrs = int(re.search(r"(\d+)\s*year", txt).group(1))
        except Exception:
            pass
        if "month" in txt:
            try:
                mos = int(re.search(r"(\d+)\s*month", txt).group(1))
            except Exception:
                pass
    else:
        try:
            yrs = int(float(txt))
        except Exception:
            pass
    return yrs + mos / 12

print("\n🧹 PREPARING RESALE DATA …")
resale["month"]                 = pd.to_datetime(resale["month"])
resale["year"]                  = resale["month"].dt.year
resale["remaining_lease_years"] = resale["remaining_lease"].apply(lease_str_to_years)

# flat age – best effort from any lease‑commence column
age_done = False
for col in ("lease_commence_date","lease_commencement_date","lease_start_date"):
    if col in resale.columns:
        resale["flat_age"] = 2025 - resale[col]
        age_done = True
        print(f"✓ flat age derived from column '{col}'")
        break
if not age_done:
    resale["flat_age"] = 99 - resale["remaining_lease_years"]
    print("✓ flat age approximated from remaining lease (99‑yr assumption)")

resale["price_per_sqm"] = resale["resale_price"] / resale["floor_area_sqm"]

# ------------------------------------------------------------
# 6️⃣  INVESTMENT CRITERIA (your exact requirements)
# ------------------------------------------------------------
criteria = {
    "price_min"          : 600_000,
    "price_max_budget"   : 700_000,     # strict budget
    "price_max_analysis" : 800_000,     # broader view
    "flat_type"          : "4 ROOM",
    "lease_min"          : 70,
    "max_flat_age"       : 20,
    "transaction_months" : 18,
    "exit_years"         : 5
}
print("\n🎯 YOUR INVESTMENT CRITERIA")
for k, v in criteria.items():
    print(f"   {k}: {v}")

cutoff_date = datetime(2025,9,1) - timedelta(days=30.44*criteria["transaction_months"])
print(f"\n📅 Transaction cutoff (last {criteria['transaction_months']} mo): {cutoff_date:%b %Y}")

# ------------------------------------------------------------
# 7️⃣  FILTER RESALE DATA STEP‑BY‑STEP
# ------------------------------------------------------------
filtered = resale.copy()
print(f"\n📊 FILTERING PROCESS – start with {len(filtered):,} rows")

# 1️⃣ recent transactions
filtered = filtered[filtered["month"] >= cutoff_date]
print(f"1️⃣ recent ({criteria['transaction_months']} mo) → {len(filtered):,}")

# 2️⃣ flat type
filtered = filtered[filtered["flat_type"] == criteria["flat_type"]]
print(f"2️⃣ flat type {criteria['flat_type']} → {len(filtered):,}")

# 3️⃣ lease
filtered = filtered[filtered["remaining_lease_years"] >= criteria["lease_min"]]
print(f"3️⃣ lease ≥{criteria['lease_min']} y → {len(filtered):,}")

# 4️⃣ age
filtered = filtered[filtered["flat_age"] <= criteria["max_flat_age"]]
print(f"4️⃣ age ≤{criteria['max_flat_age']} y → {len(filtered):,}")

# split into strict‑budget and broader analysis
budget_df   = filtered[(filtered["resale_price"] >= criteria["price_min"]) &
                      (filtered["resale_price"] <= criteria["price_max_budget"])].copy()

analysis_df = filtered[(filtered["resale_price"] >= criteria["price_min"]) &
                      (filtered["resale_price"] <= criteria["price_max_analysis"])].copy()

print(f"5️⃣ budget (${criteria['price_min']:,}-${criteria['price_max_budget']:,}) → {len(budget_df):,}")
print(f"6️⃣ analysis (${criteria['price_min']:,}-${criteria['price_max_analysis']:,}) → {len(analysis_df):,}")

# ------------------------------------------------------------
# 8️⃣  LOCATION TIERING (centrality + MRT)
# ------------------------------------------------------------
def build_location_lookup():
    tiers = {
        "Prime Central (CBD Core)" : {
            "towns"      : ["DOWNTOWN CORE","MARINA EAST","MARINA SOUTH","MUSEUM",
                           "NEWTON","ORCHARD","OUTRAM","RIVER VALLEY","ROCHOR"],
            "score"      : 10, "transport":"Multiple MRT lines, CBD core", "centrality":"Prime Central"
        },
        "Inner Central Ring" : {
            "towns"      : ["BUKIT MERAH","KALLANG/WHAMPOA","QUEENSTOWN","TOA PAYOH",
                           "MARINE PARADE","GEYLANG","NOVENA"],
            "score"      : 9,  "transport":"Direct CBD access, established MRT", "centrality":"Inner Central"
        },
        "Central Well‑Connected" : {
            "towns"      : ["BISHAN","ANG MO KIO","CLEMENTI","REDHILL","TIONG BAHRU"],
            "score"      : 8,  "transport":"Major MRT interchange", "centrality":"Central Ring"
        },
        "Mature Transport Hubs" : {
            "towns"      : ["BEDOK","TAMPINES","JURONG EAST","HOUGANG","SERANGOON",
                           "BUKIT BATOK","BUKIT PANJANG","CHOA CHU KANG"],
            "score"      : 7,  "transport":"Established MRT lines", "centrality":"Mature Outer"
        },
        "Well‑Connected Outer" : {
            "towns"      : ["PASIR RIS","JURONG WEST","WOODLANDS","YISHUN","ADMIRALTY"],
            "score"      : 6,  "transport":"Single MRT line", "centrality":"Outer Ring"
        },
        "Developing Areas" : {
            "towns"      : ["SEMBAWANG","PUNGGOL","SENGKANG"],
            "score"      : 5,  "transport":"Newer MRT lines", "centrality":"New Development"
        }
    }
    lookup = {}
    for tier,info in tiers.items():
        for t in info["towns"]:
            lookup[t.upper()] = (tier, info["score"], info["transport"], info["centrality"])
    return lookup

loc_lookup = build_location_lookup()

def get_location_info(town):
    key = str(town).upper()
    return loc_lookup.get(key, ("Other Areas",4,"Limited transport","Peripheral"))

# enrich both data‑sets
for df in (budget_df, analysis_df):
    info = df["town"].apply(get_location_info)
    df["location_tier"] = [i[0] for i in info]
    df["location_score"] = [i[1] for i in info]
    df["transport_info"] = [i[2] for i in info]
    df["centrality"] = [i[3] for i in info]

# ------------------------------------------------------------
# 9️⃣  NOISE FILTER – avoid main‑road / directly beside MRT
# ------------------------------------------------------------
NOISE_KEYWORDS = [
    "HIGHWAY","EXPRESSWAY","PIE","CTE","AYE","BKE","SLE","TPE","KJE","MCE","ECP",
    "MAIN ROAD","MAJOR ROAD","ARTERIAL"
]
MRT_KEYWORDS = ["MRT","STATION","INTERCHANGE","TERMINAL"]

def noise_filter(df):
    df["high_noise"] = df["street_name"].apply(
        lambda x: any(k in str(x).upper() for k in NOISE_KEYWORDS) if pd.notna(x) else False)
    df["mrt_near"]   = df["street_name"].apply(
        lambda x: any(k in str(x).upper() for k in MRT_KEYWORDS) if pd.notna(x) else False)
    df["noise_flag"] = df["high_noise"] | df["mrt_near"]
    kept = df[~df["noise_flag"]].copy()
    print(f"🔇 Noise filter → removed {len(df)-len(kept):,} rows; kept {len(kept):,}")
    return kept.drop(columns=["high_noise","mrt_near","noise_flag"])

budget_df   = noise_filter(budget_df)
analysis_df = noise_filter(analysis_df)

# ------------------------------------------------------------
# 1️⃣0️⃣  SCORING (size, price‑efficiency, lease, age, location)
# ------------------------------------------------------------
def norm(series, reverse=False):
    if series.empty: return series
    mn, mx = series.min(), series.max()
    if mx == mn:   return pd.Series(50, index=series.index)
    if reverse:    return 100 * (mx - series) / (mx - mn)
    return 100 * (series - mn) / (mx - mn)

def score(df):
    df["size_score"]          = norm(df["floor_area_sqm"])
    df["price_eff_score"]     = norm(df["price_per_sqm"], reverse=True)
    df["lease_score"]         = norm(df["remaining_lease_years"])
    df["age_score"]           = norm(df["flat_age"], reverse=True)   # newer is better
    df["loc_score_norm"]      = df["location_score"] * 10            # 0‑100 scale

    w = {"loc":0.40, "price":0.25, "size":0.20, "lease":0.10, "age":0.05}
    df["overall_score"] = (
        w["loc"]   * df["loc_score_norm"] +
        w["price"] * df["price_eff_score"] +
        w["size"]  * df["size_score"] +
        w["lease"] * df["lease_score"] +
        w["age"]   * df["age_score"]
    ).round(2)

    df["full_address"] = ("Block " + df["block"].astype(str) + " " +
                          df["street_name"].astype(str) + ", " + df["town"])
    return df

budget_df   = score(budget_df)
analysis_df = score(analysis_df)

# ------------------------------------------------------------
# 1️⃣1️⃣  FINAL RECOMMENDATIONS
# ------------------------------------------------------------
def show_top(df, n, title):
    print(f"\n🏆 {title} – TOP {n}")
    top = df.nlargest(n, "overall_score")
    for i, (_, r) in enumerate(top.iterrows(), 1):
        geo = "🗺️ Geometry" if not pd.isna(r.get("geometry")) else "📍 Address only"
        print(f"\n{i:2d}. {r['full_address']}")
        print(f"   {r['centrality']} | {r['location_tier']} | {geo}")
        print(f"   ${r['resale_price']:,.0f} | {r['floor_area_sqm']:.0f} sqm | ${r['price_per_sqm']:,.0f}/sqm")
        print(f"   Lease {r['remaining_lease_years']:.1f} y | Age {r['flat_age']:.0f} y")
        print(f"   {r['transport_info']}")
        print(f"   Score {r['overall_score']:.1f}/100 | {r['month'].strftime('%b %Y')}")
    return top

top_budget   = show_top(budget_df,   15, "YOUR BUDGET RECOMMENDATIONS")
top_analysis = show_top(analysis_df, 20, "BROADER ANALYSIS RECOMMENDATIONS")

# ------------------------------------------------------------
# 1️⃣2️⃣  TOWN‑LEVEL OVERVIEW (analysis set)
# ------------------------------------------------------------
print("\n🏘️ TOWN OVERVIEW (analysis set)")
if not analysis_df.empty:
    town_stats = (analysis_df.groupby(['town','centrality','location_tier'])
                         .agg(units=('resale_price','count'),
                              avg_price=('resale_price','mean'),
                              avg_sqm=('floor_area_sqm','mean'),
                              avg_ppsqm=('price_per_sqm','mean'),
                              avg_score=('overall_score','mean'))
                         .reset_index()
                         .sort_values('avg_score',ascending=False))
    print(f"{'Rank':>4} {'Town':20} {'Centrality':15} {'Units':>5} {'Avg Price':>11} {'Avg Size':>9} {'Score':>6}")
    print("-"*80)
    for i, row in town_stats.head(15).iterrows():
        print(f"{i+1:4d} {row['town']:20} {row['centrality']:15} {row['units']:5d} ${row['avg_price']:10.0f} {row['avg_sqm']:9.1f} {row['avg_score']:6.1f}")

# ------------------------------------------------------------
# 1️⃣3️⃣  4‑ROOM MARKET OVERVIEW (last 24 months)
# ------------------------------------------------------------
print("\n📈 4‑ROOM MARKET OVERVIEW (last 24 mo)")
cutoff_market = datetime(2025,9,1) - timedelta(days=30.44*24)
market_4room = resale[(resale["flat_type"]=="4 ROOM") & (resale["month"]>=cutoff_market)].copy()
if not market_4room.empty:
    market_4room["centrality"] = market_4room["town"].apply(lambda t: get_location_info(t)[3])
    bands = [("Under $500k",0,500_000),
             ("$500k‑$600k",500_000,600_000),
             ("$600k‑$700k",600_000,700_000),
             ("$700k‑$800k",700_000,800_000),
             ("$800k‑$900k",800_000,900_000),
             ("$900k‑$1M",900_000,1_000_000),
             ("Over $1M",1_000_000,float('inf'))]
    print(f"{'Band':15} {'Count':>6} {'%':>6} {'Towns':>6} {'Avg Size':>9}")
    for name, lo, hi in bands:
        if hi==float('inf'):
            dfb = market_4room[market_4room["resale_price"]>=lo]
        else:
            dfb = market_4room[(market_4room["resale_price"]>=lo) & (market_4room["resale_price"]<hi)]
        if not dfb.empty:
            pct = len(dfb)/len(market_4room)*100
            towns = dfb["town"].nunique()
            avgsize = dfb["floor_area_sqm"].mean()
            print(f"{name:15} {len(dfb):6,} {pct:5.1f}% {towns:6d} {avgsize:8.0f}sqm")

# ------------------------------------------------------------
# 1️⃣4️⃣  EXPORT CSVs
# ------------------------------------------------------------
print("\n💾 EXPORTING CSV FILES …")
export_cols = ['full_address','town','block','street_name','flat_type',
               'resale_price','floor_area_sqm','price_per_sqm',
               'remaining_lease_years','flat_age','location_tier',
               'centrality','transport_info','overall_score','month']

if 'geometry' in analysis_df.columns:
    export_cols.append('geometry')

try:
    if not top_budget.empty:
        top_budget[export_cols].to_csv('hdb_budget_top15.csv', index=False)
        print("✅ hdb_budget_top15.csv")
    if not top_analysis.empty:
        top_analysis[export_cols].to_csv('hdb_analysis_top20.csv', index=False)
        print("✅ hdb_analysis_top20.csv")
    if not analysis_df.empty:
        analysis_df[export_cols].to_csv('hdb_all_analysis_filtered.csv', index=False)
        print("✅ hdb_all_analysis_filtered.csv")
except Exception as e:
    print("⚠️  CSV export error:", e)

# ------------------------------------------------------------
# FINAL EXECUTIVE SUMMARY
# ------------------------------------------------------------
print("\n" + "="*80)
print("📋 EXECUTIVE SUMMARY")
print("="*80)

if not budget_df.empty:
    print(f"\n🔎 Budget window (${criteria['price_min']:,}-${criteria['price_max_budget']:,}):")
    print(f"   • Units found            : {len(budget_df):,}")
    print(f"   • Towns represented      : {budget_df['town'].nunique()}")
    print(f"   • Avg size (sqm)         : {budget_df['floor_area_sqm'].mean():.1f}")
    print(f"   • Price range            : ${budget_df['resale_price'].min():,.0f} – ${budget_df['resale_price'].max():,.0f}")
else:
    print("\n⚠️ No properties matched the strict budget after all filters.")

if 'large_3room' in locals() and not large_3room.empty:
    print(f"\n🏠 Large 3‑room alternatives (size ≥ median 4‑room, price ≤ ${criteria['price_max_budget']:,})")
    print(f"   • Count : {len(large_3room):,}")
    print(f"   • Avg price : ${large_3room['resale_price'].mean():,.0f}")

print("\n✅ Analysis complete – check the generated CSV files for full details.")


🏠 COMPLETE HDB ANALYSIS WITH POLYGON DATA

Columns after normalising
Resale : ['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_date', 'remaining_lease', 'resale_price']
Blocks : ['name', 'description', 'geometry']

🔍 TESTING DESCRIPTION EXTRACTION (first 3 rows)

Row 0 – description snippet:
<center><table><tr><th colspan='2' align='center'><em>Attributes</em></th></tr><tr bgcolor="#E3E3F3"> <th>BLK_NO</th> <t...
Extracted → {'block': '780C', 'street_name': None, 'town': 'ATTRIBUTES   BLK_NO   780C     ST_COD   WOC05L     ENTITYID   1991     POSTAL_COD   733780     INC_CRC   B93627FDFF3D6313     FMEL_UPD_D   20230328181025'}

Row 1 – description snippet:
<center><table><tr><th colspan='2' align='center'><em>Attributes</em></th></tr><tr bgcolor="#E3E3F3"> <th>BLK_NO</th> <t...
Extracted → {'block': '373', 'street_name': None, 'town': 'ATTRIBUTES   BLK_NO   373     ST_COD   BUS09S     ENTITYID   6782     POSTAL_COD   6

## 

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

# ------------------------------------------------------------
# 1. DATA PREPARATION & FILTERING
# ------------------------------------------------------------

def parse_remaining_lease(lease_str):
    """Convert remaining lease string to years (float)"""
    if pd.isna(lease_str):
        return np.nan
    
    lease_str = str(lease_str).strip()
    
    # Handle different formats: "95 years 03 months", "95 years", "95"
    years = 0
    months = 0
    
    # Extract years
    if 'year' in lease_str.lower():
        year_match = lease_str.lower().split('year')[0].strip()
        try:
            years = int(year_match.split()[-1])
        except:
            years = 0
    
    # Extract months
    if 'month' in lease_str.lower():
        parts = lease_str.lower().split('year')
        if len(parts) > 1:
            month_part = parts[1].strip()
            month_match = month_part.split('month')[0].strip()
            try:
                months = int(month_match.split()[-1])
            except:
                months = 0
    
    return years + months/12

# Clean and prepare data
print("=== DATA PREPARATION ===")
print("Available columns:", list(resale.columns))

resale['remaining_lease_years'] = resale['remaining_lease'].apply(parse_remaining_lease)
resale['month'] = pd.to_datetime(resale['month'])
resale['year'] = resale['month'].dt.year

# Calculate age of flat (2024 - lease_commence_date)
# Check for the correct column name
lease_start_col = None
possible_lease_cols = ['lease_commence_date', 'lease_commencement_date', 'lease_start_date']

for col in possible_lease_cols:
    if col in resale.columns:
        lease_start_col = col
        break

if lease_start_col:
    current_year = 2024
    resale['flat_age'] = current_year - resale[lease_start_col]
    print(f"Using column '{lease_start_col}' for lease commencement")
else:
    # If no lease start column found, calculate from remaining lease
    # Assume original lease is 99 years
    original_lease = 99
    resale['flat_age'] = original_lease - resale['remaining_lease_years']
    print("No lease commencement column found, calculating from remaining lease")

print(f"Original dataset: {len(resale):,} records")

# ------------------------------------------------------------
# 2. APPLY YOUR CRITERIA
# ------------------------------------------------------------

# Your criteria
criteria = {
    'price_min': 600000,
    'price_max': 800000,
    'flat_type': '4 ROOM',
    'remaining_lease_min': 70,
    'flat_age_max': 20  # Used for max 20 years
}

# Apply filters
filtered_data = resale.copy()

# 1. Price range
filtered_data = filtered_data[
    (filtered_data['resale_price'] >= criteria['price_min']) & 
    (filtered_data['resale_price'] <= criteria['price_max'])
]
print(f"After price filter (${criteria['price_min']:,}-${criteria['price_max']:,}): {len(filtered_data):,} records")

# 2. 4-room flats
filtered_data = filtered_data[filtered_data['flat_type'] == criteria['flat_type']]
print(f"After 4-room filter: {len(filtered_data):,} records")

# 3. Remaining lease >= 70 years
filtered_data = filtered_data[filtered_data['remaining_lease_years'] >= criteria['remaining_lease_min']]
print(f"After remaining lease >= {criteria['remaining_lease_min']} years: {len(filtered_data):,} records")

# 4. Flat age <= 20 years
filtered_data = filtered_data[filtered_data['flat_age'] <= criteria['flat_age_max']]
print(f"After flat age <= {criteria['flat_age_max']} years: {len(filtered_data):,} records")

# Only keep recent data (last 2 years) for current market prices
recent_data = filtered_data[filtered_data['year'] >= 2022]
print(f"Recent transactions (2022+): {len(recent_data):,} records")

# ------------------------------------------------------------
# 3. CENTRAL LOCATION SCORING (Without API)
# ------------------------------------------------------------

# Define central areas based on Singapore planning areas
# This is based on general knowledge of Singapore geography
central_areas = {
    'CBD/Central': ['DOWNTOWN CORE', 'MARINA EAST', 'MARINA SOUTH', 'MUSEUM', 'NEWTON', 'ORCHARD', 'OUTRAM', 'RIVER VALLEY', 'ROCHOR', 'SINGAPORE RIVER', 'STRAITS VIEW'],
    'Central-Adjacent': ['BUKIT MERAH', 'KALLANG/WHAMPOA', 'QUEENSTOWN', 'TOA PAYOH', 'BISHAN', 'MARINE PARADE'],
    'Well-Connected': ['ANG MO KIO', 'BEDOK', 'CLEMENTI', 'GEYLANG', 'HOUGANG', 'TAMPINES', 'JURONG EAST', 'BUKIT BATOK', 'BUKIT PANJANG', 'CHOA CHU KANG'],
    'Suburban': ['JURONG WEST', 'WOODLANDS', 'YISHUN', 'SEMBAWANG', 'PASIR RIS', 'PUNGGOL', 'SENGKANG', 'SERANGOON']
}

def get_centrality_score(town):
    """Assign centrality score: 4=most central, 1=most suburban"""
    for category, towns in central_areas.items():
        if town.upper() in [t.upper() for t in towns]:
            if category == 'CBD/Central':
                return 4
            elif category == 'Central-Adjacent':
                return 3
            elif category == 'Well-Connected':
                return 2
            else:
                return 1
    return 1  # Default to suburban if not found

recent_data = recent_data.copy()
recent_data['centrality_score'] = recent_data['town'].apply(get_centrality_score)

# ------------------------------------------------------------
# 4. COMPREHENSIVE ANALYSIS
# ------------------------------------------------------------

print("\n=== ANALYSIS RESULTS ===")

# 4a. Price Analysis by Town
print("\n--- PRICE ANALYSIS BY TOWN ---")
town_analysis = recent_data.groupby('town').agg({
    'resale_price': ['mean', 'median', 'count', 'std'],
    'floor_area_sqm': ['mean', 'median'],
    'remaining_lease_years': 'mean',
    'flat_age': 'mean',
    'centrality_score': 'first'
}).round(2)

# Flatten column names
town_analysis.columns = ['avg_price', 'median_price', 'count', 'price_std', 
                        'avg_sqm', 'median_sqm', 'avg_remaining_lease', 
                        'avg_flat_age', 'centrality_score']

# Calculate price per sqm
town_analysis['price_per_sqm'] = (town_analysis['avg_price'] / town_analysis['avg_sqm']).round(2)

# Sort by centrality score (descending) then by price
town_analysis = town_analysis.sort_values(['centrality_score', 'avg_price'], ascending=[False, True])

print("Top locations by centrality and value:")
print(town_analysis.head(15)[['avg_price', 'median_price', 'count', 'avg_sqm', 'price_per_sqm', 'centrality_score']])

# 4b. Value Analysis - Best Deals
print("\n--- BEST VALUE ANALYSIS ---")

# Calculate value metrics
recent_data = recent_data.copy()
recent_data['price_per_sqm'] = recent_data['resale_price'] / recent_data['floor_area_sqm']

# Create composite score: higher is better value
# Normalize scores to 0-100 scale
def normalize_score(series, reverse=False):
    """Normalize to 0-100, reverse=True for metrics where lower is better"""
    min_val, max_val = series.min(), series.max()
    if reverse:
        return 100 * (max_val - series) / (max_val - min_val)
    else:
        return 100 * (series - min_val) / (max_val - min_val)

recent_data['sqm_score'] = normalize_score(recent_data['floor_area_sqm'])  # Bigger is better
recent_data['price_score'] = normalize_score(recent_data['price_per_sqm'], reverse=True)  # Lower price/sqm is better
recent_data['lease_score'] = normalize_score(recent_data['remaining_lease_years'])  # More lease is better
recent_data['centrality_normalized'] = (recent_data['centrality_score'] - 1) / 3 * 100  # Convert to 0-100

# Weighted composite score (you can adjust weights)
weights = {
    'centrality': 0.3,  # 30% weight to location
    'price': 0.25,      # 25% weight to value for money
    'size': 0.25,       # 25% weight to size
    'lease': 0.2        # 20% weight to remaining lease
}

recent_data['value_score'] = (
    weights['centrality'] * recent_data['centrality_normalized'] +
    weights['price'] * recent_data['price_score'] +
    weights['size'] * recent_data['sqm_score'] +
    weights['lease'] * recent_data['lease_score']
)

# Top deals
print("Best overall value properties (Top 20):")
best_deals = recent_data.nlargest(20, 'value_score')[
    ['town', 'block', 'street_name', 'resale_price', 'floor_area_sqm', 
     'price_per_sqm', 'remaining_lease_years', 'centrality_score', 'value_score', 'month']
]
print(best_deals.to_string(index=False))

# 4c. Size Comparison with 3-Room Flats
print("\n--- SIZE COMPARISON: 3-ROOM vs 4-ROOM FLATS ---")

# Get 3-room data for comparison
three_room_data = resale[
    (resale['flat_type'] == '3 ROOM') & 
    (resale['year'] >= 2022) &
    (resale['flat_age'] <= 30)  # Allow older 3-room for comparison
].copy()

size_comparison = pd.DataFrame({
    'Flat Type': ['3-Room (Older)', '4-Room (Your Criteria)'],
    'Avg Size (sqm)': [
        three_room_data['floor_area_sqm'].mean(),
        recent_data['floor_area_sqm'].mean()
    ],
    'Median Size (sqm)': [
        three_room_data['floor_area_sqm'].median(),
        recent_data['floor_area_sqm'].median()
    ],
    'Avg Price': [
        three_room_data['resale_price'].mean(),
        recent_data['resale_price'].mean()
    ],
    'Count': [len(three_room_data), len(recent_data)]
}).round(2)

print(size_comparison)

# Check if older 3-room flats are larger than new 4-room
older_3room_larger = three_room_data[
    three_room_data['floor_area_sqm'] > recent_data['floor_area_sqm'].median()
]
print(f"\nOlder 3-room flats larger than median 4-room: {len(older_3room_larger):,} properties")

# 4d. Investment Perspective (5-year hold)
print("\n--- 5-YEAR INVESTMENT ANALYSIS ---")

# Price trends over recent years
price_trends = recent_data.groupby(['town', 'year'])['price_per_sqm'].mean().reset_index()

# Calculate price appreciation for towns with sufficient data
town_appreciation = {}
for town in recent_data['town'].unique():
    town_data = price_trends[price_trends['town'] == town]
    if len(town_data) >= 2:
        early_price = town_data[town_data['year'] == town_data['year'].min()]['price_per_sqm'].iloc[0]
        recent_price = town_data[town_data['year'] == town_data['year'].max()]['price_per_sqm'].iloc[0]
        years_diff = town_data['year'].max() - town_data['year'].min()
        if years_diff > 0:
            annual_appreciation = ((recent_price / early_price) ** (1/years_diff) - 1) * 100
            town_appreciation[town] = annual_appreciation

appreciation_df = pd.DataFrame(list(town_appreciation.items()), 
                              columns=['town', 'annual_appreciation_%'])
appreciation_df = appreciation_df.sort_values('annual_appreciation_%', ascending=False)

print("Price appreciation by town (annual %):")
print(appreciation_df.head(10))

# ------------------------------------------------------------
# 5. FINAL RECOMMENDATIONS
# ------------------------------------------------------------

print("\n=== RECOMMENDATIONS ===")

# Top 5 recommendations based on your criteria
print("\n--- TOP 5 RECOMMENDED AREAS ---")
final_recommendations = town_analysis.head(5).copy()
final_recommendations = final_recommendations.merge(
    appreciation_df, on='town', how='left'
)

for i, (town, data) in enumerate(final_recommendations.iterrows(), 1):
    print(f"\n{i}. {town}")
    print(f"   Average Price: ${data['avg_price']:,.0f}")
    print(f"   Average Size: {data['avg_sqm']:.1f} sqm")
    print(f"   Price/sqm: ${data['price_per_sqm']:,.0f}")
    print(f"   Centrality Score: {data['centrality_score']}/4")
    print(f"   Available Properties: {data['count']:.0f}")
    if not pd.isna(data.get('annual_appreciation_%')):
        print(f"   Recent Price Growth: {data['annual_appreciation_%']:.1f}% per year")

# Specific property recommendations
print(f"\n--- SPECIFIC PROPERTY RECOMMENDATIONS ---")
print("Based on highest value scores:")

top_properties = recent_data.nlargest(10, 'value_score')
for i, (_, prop) in enumerate(top_properties.iterrows(), 1):
    print(f"\n{i}. Block {prop['block']} {prop['street_name']}, {prop['town']}")
    print(f"   Price: ${prop['resale_price']:,.0f}")
    print(f"   Size: {prop['floor_area_sqm']:.0f} sqm (${prop['price_per_sqm']:,.0f}/sqm)")
    print(f"   Remaining Lease: {prop['remaining_lease_years']:.1f} years")
    print(f"   Transaction Date: {prop['month'].strftime('%Y-%m')}")
    print(f"   Value Score: {prop['value_score']:.1f}/100")

print(f"\n=== SUMMARY STATISTICS ===")
print(f"Total properties meeting your criteria: {len(recent_data):,}")
print(f"Price range: ${recent_data['resale_price'].min():,.0f} - ${recent_data['resale_price'].max():,.0f}")
print(f"Average price: ${recent_data['resale_price'].mean():,.0f}")
print(f"Size range: {recent_data['floor_area_sqm'].min():.0f} - {recent_data['floor_area_sqm'].max():.0f} sqm")
print(f"Average size: {recent_data['floor_area_sqm'].mean():.1f} sqm")
print(f"Towns available: {recent_data['town'].nunique()}")

# Save processed data for further analysis
recent_data.to_csv('hdb_analysis_results.csv', index=False)
town_analysis.to_csv('hdb_town_analysis.csv')
print(f"\nDetailed results saved to 'hdb_analysis_results.csv' and 'hdb_town_analysis.csv'")

=== DATA PREPARATION ===
Available columns: ['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_date', 'remaining_lease', 'resale_price', 'year', 'remaining_lease_years', 'flat_age', 'price_per_sqm']
Using column 'lease_commence_date' for lease commencement
Original dataset: 215,121 records
After price filter ($600,000-$800,000): 45,024 records
After 4-room filter: 17,101 records
After remaining lease >= 70 years: 14,113 records
After flat age <= 20 years: 11,309 records
Recent transactions (2022+): 8,220 records

=== ANALYSIS RESULTS ===

--- PRICE ANALYSIS BY TOWN ---
Top locations by centrality and value:
                 avg_price  median_price  count  avg_sqm  price_per_sqm  \
town                                                                      
QUEENSTOWN       740867.47      750000.0     66    89.50        8277.85   
KALLANG/WHAMPOA  747643.08      755000.0    185    90.72        8241.22   
TOA PAYOH        

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set up plotting
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("="*60)
print("🏠 HDB 4-ROOM FLAT INVESTMENT ANALYSIS")
print("="*60)

# ------------------------------------------------------------
# 1. DATA PREPARATION & EXPLORATION
# ------------------------------------------------------------
def explore_data_structure():
    """Explore the dataset structure first"""
    print("\n📊 DATA EXPLORATION")
    print("-" * 40)
    print(f"Dataset shape: {resale.shape}")
    print(f"Columns: {list(resale.columns)}")
    print(f"\nSample data:")
    print(resale.head(3))
    print(f"\nData types:")
    print(resale.dtypes)
    
    # Check for missing values
    missing = resale.isnull().sum()
    if missing.sum() > 0:
        print(f"\nMissing values:")
        print(missing[missing > 0])

explore_data_structure()

# ------------------------------------------------------------
# 2. PARSE REMAINING LEASE
# ------------------------------------------------------------
def parse_remaining_lease(lease_str):
    """Convert remaining lease string to years (float)"""
    if pd.isna(lease_str):
        return np.nan
    
    lease_str = str(lease_str).strip().lower()
    
    years = 0
    months = 0
    
    # Handle different formats
    if 'year' in lease_str:
        parts = lease_str.split('year')
        try:
            years = int(parts[0].strip().split()[-1])
        except:
            years = 0
            
        # Check for months after years
        if len(parts) > 1 and 'month' in parts[1]:
            try:
                months_part = parts[1].split('month')[0].strip()
                months = int(months_part.split()[-1])
            except:
                months = 0
    else:
        # If no 'years' keyword, assume it's just a number
        try:
            years = int(float(lease_str))
        except:
            years = 0
    
    return years + months/12

# ------------------------------------------------------------
# 3. CLEAN AND PREPARE DATA
# ------------------------------------------------------------
print("\n🧹 DATA CLEANING & PREPARATION")
print("-" * 40)

# Parse dates and remaining lease
resale['month'] = pd.to_datetime(resale['month'])
resale['year'] = resale['month'].dt.year
resale['remaining_lease_years'] = resale['remaining_lease'].apply(parse_remaining_lease)

# Calculate flat age - try different possible column names
age_calculated = False
lease_cols = ['lease_commence_date', 'lease_commencement_date', 'lease_start_date']

for col in lease_cols:
    if col in resale.columns:
        current_year = 2024
        resale['flat_age'] = current_year - resale[col]
        print(f"✓ Calculated flat age using: {col}")
        age_calculated = True
        break

if not age_calculated:
    # Calculate from remaining lease (assuming 99-year lease)
    resale['flat_age'] = 99 - resale['remaining_lease_years']
    print("✓ Calculated flat age from remaining lease (99-year assumption)")

# Calculate price per sqm
resale['price_per_sqm'] = resale['resale_price'] / resale['floor_area_sqm']

print(f"Original records: {len(resale):,}")

# ------------------------------------------------------------
# 4. APPLY YOUR INVESTMENT CRITERIA
# ------------------------------------------------------------
print("\n🎯 APPLYING INVESTMENT CRITERIA")
print("-" * 40)

# Your specific requirements
criteria = {
    'price_min': 600_000,
    'price_max': 800_000,      # Extended for better picture
    'flat_type': '4 ROOM',
    'remaining_lease_min': 70,
    'max_flat_age': 20,
    'recent_years': 2          # Last 2 years for current market
}

print("Investment Criteria:")
for key, value in criteria.items():
    print(f"  • {key}: {value}")

# Step-by-step filtering
filtered = resale.copy()

# 1. Price range
filtered = filtered[
    (filtered['resale_price'] >= criteria['price_min']) & 
    (filtered['resale_price'] <= criteria['price_max'])
]
print(f"\n1️⃣ After price filter (${criteria['price_min']:,}-${criteria['price_max']:,}): {len(filtered):,}")

# 2. 4-room flats only
filtered = filtered[filtered['flat_type'] == criteria['flat_type']]
print(f"2️⃣ After 4-room filter: {len(filtered):,}")

# 3. Remaining lease >= 70 years
filtered = filtered[filtered['remaining_lease_years'] >= criteria['remaining_lease_min']]
print(f"3️⃣ After remaining lease ≥{criteria['remaining_lease_min']} years: {len(filtered):,}")

# 4. Flat age <= 20 years (houses used for max 20 years)
filtered = filtered[filtered['flat_age'] <= criteria['max_flat_age']]
print(f"4️⃣ After flat age ≤{criteria['max_flat_age']} years: {len(filtered):,}")

# 5. Recent transactions only
recent_cutoff = datetime.now() - timedelta(days=365 * criteria['recent_years'])
recent = filtered[filtered['month'] >= recent_cutoff]
print(f"5️⃣ Recent transactions (last {criteria['recent_years']} years): {len(recent):,}")

if len(recent) == 0:
    print("⚠️ No recent transactions found, using last 3 years data")
    recent_cutoff = datetime.now() - timedelta(days=365 * 3)
    recent = filtered[filtered['month'] >= recent_cutoff]
    print(f"📊 Using last 3 years: {len(recent):,} transactions")

# ------------------------------------------------------------
# 5. LOCATION ANALYSIS (CENTRALITY SCORING)
# ------------------------------------------------------------
print("\n📍 LOCATION & CENTRALITY ANALYSIS")
print("-" * 40)

# Singapore regions by centrality (without using APIs)
location_tiers = {
    'Prime Central': {
        'towns': ['DOWNTOWN CORE', 'MARINA EAST', 'MARINA SOUTH', 'MUSEUM', 
                 'NEWTON', 'ORCHARD', 'OUTRAM', 'RIVER VALLEY', 'ROCHOR'],
        'score': 10,
        'description': 'CBD and prime districts'
    },
    'Central Core': {
        'towns': ['BUKIT MERAH', 'KALLANG/WHAMPOA', 'QUEENSTOWN', 'TOA PAYOH', 
                 'MARINE PARADE', 'GEYLANG'],
        'score': 9,
        'description': 'Central areas with excellent connectivity'
    },
    'Well-Connected': {
        'towns': ['BISHAN', 'ANG MO KIO', 'CLEMENTI', 'BEDOK', 'TAMPINES',
                 'JURONG EAST', 'HOUGANG'],
        'score': 8,
        'description': 'Major transport hubs and developed towns'
    },
    'Established': {
        'towns': ['BUKIT BATOK', 'BUKIT PANJANG', 'CHOA CHU KANG', 'SERANGOON',
                 'PASIR RIS', 'JURONG WEST'],
        'score': 7,
        'description': 'Mature estates with good amenities'
    },
    'Developing': {
        'towns': ['WOODLANDS', 'YISHUN', 'SEMBAWANG', 'PUNGGOL', 'SENGKANG'],
        'score': 6,
        'description': 'Newer/developing areas'
    }
}

def get_location_tier(town):
    """Get location tier info for a town"""
    town_upper = town.upper()
    for tier_name, tier_info in location_tiers.items():
        if town_upper in [t.upper() for t in tier_info['towns']]:
            return tier_name, tier_info['score'], tier_info['description']
    return 'Other', 5, 'Other areas'

# Apply location scoring
recent = recent.copy()
location_data = recent['town'].apply(get_location_tier)
recent['location_tier'] = [x[0] for x in location_data]
recent['location_score'] = [x[1] for x in location_data]
recent['tier_description'] = [x[2] for x in location_data]

# Show location distribution
location_summary = recent.groupby('location_tier').agg({
    'resale_price': ['mean', 'count'],
    'floor_area_sqm': 'mean',
    'price_per_sqm': 'mean',
    'location_score': 'first'
}).round(0)

location_summary.columns = ['avg_price', 'count', 'avg_sqm', 'avg_price_per_sqm', 'score']
location_summary = location_summary.sort_values('score', ascending=False)

print("📊 Properties by Location Tier:")
print(location_summary)

# ------------------------------------------------------------
# 6. COMPREHENSIVE TOWN-LEVEL ANALYSIS
# ------------------------------------------------------------
print("\n🏘️ DETAILED TOWN ANALYSIS")
print("-" * 40)

town_stats = recent.groupby('town').agg({
    'resale_price': ['mean', 'median', 'min', 'max', 'count'],
    'floor_area_sqm': ['mean', 'median', 'min', 'max'],
    'price_per_sqm': ['mean', 'median'],
    'remaining_lease_years': ['mean', 'min'],
    'flat_age': 'mean',
    'location_score': 'first',
    'location_tier': 'first'
}).round(2)

# Flatten columns
town_stats.columns = ['price_mean', 'price_median', 'price_min', 'price_max', 'count',
                     'sqm_mean', 'sqm_median', 'sqm_min', 'sqm_max',
                     'psf_mean', 'psf_median', 'lease_mean', 'lease_min',
                     'age_mean', 'location_score', 'location_tier']

# Calculate value metrics
town_stats['value_score'] = (
    (town_stats['sqm_mean'] / town_stats['sqm_mean'].max()) * 25 +  # Size value (25%)
    ((town_stats['psf_mean'].max() - town_stats['psf_mean']) / town_stats['psf_mean'].max()) * 25 +  # Price value (25%)
    (town_stats['location_score'] / 10) * 30 +  # Location (30%)
    (town_stats['lease_mean'] / town_stats['lease_mean'].max()) * 20  # Remaining lease (20%)
).round(2)

# Sort by value score
town_stats = town_stats.sort_values(['location_score', 'value_score'], ascending=[False, False])

print("🏆 TOP TOWNS BY VALUE:")
top_towns = town_stats.head(15)[['price_mean', 'count', 'sqm_mean', 'psf_mean', 
                                'location_score', 'value_score', 'location_tier']]
print(top_towns)

# ------------------------------------------------------------
# 7. 3-ROOM vs 4-ROOM SIZE COMPARISON
# ------------------------------------------------------------
print("\n📏 3-ROOM vs 4-ROOM SIZE COMPARISON")
print("-" * 40)

# Get 3-room data for comparison
three_room = resale[
    (resale['flat_type'] == '3 ROOM') & 
    (resale['year'] >= 2022) &
    (resale['flat_age'] <= 30)  # Allow older 3-room for size comparison
].copy()

# Size comparison analysis
size_analysis = pd.DataFrame({
    'Property Type': ['3-Room Flats (≤30 years old)', '4-Room Flats (Your Criteria)'],
    'Count': [len(three_room), len(recent)],
    'Avg Size (sqm)': [three_room['floor_area_sqm'].mean(), recent['floor_area_sqm'].mean()],
    'Median Size (sqm)': [three_room['floor_area_sqm'].median(), recent['floor_area_sqm'].median()],
    'Size Range': [f"{three_room['floor_area_sqm'].min():.0f}-{three_room['floor_area_sqm'].max():.0f}", 
                   f"{recent['floor_area_sqm'].min():.0f}-{recent['floor_area_sqm'].max():.0f}"],
    'Avg Price': [three_room['resale_price'].mean(), recent['resale_price'].mean()],
    'Avg Price/sqm': [three_room['price_per_sqm'].mean(), recent['price_per_sqm'].mean()]
}).round(2)

print(size_analysis)

# Find 3-room flats larger than median 4-room
median_4room_size = recent['floor_area_sqm'].median()
large_3room = three_room[three_room['floor_area_sqm'] > median_4room_size]

print(f"\n📊 3-room flats larger than median 4-room ({median_4room_size:.0f} sqm): {len(large_3room):,}")
if len(large_3room) > 0:
    print(f"   Average size of large 3-rooms: {large_3room['floor_area_sqm'].mean():.1f} sqm")
    print(f"   Average price: ${large_3room['resale_price'].mean():,.0f}")

# ------------------------------------------------------------
# 8. INVESTMENT ANALYSIS (5-YEAR PERSPECTIVE)
# ------------------------------------------------------------
print("\n💰 5-YEAR INVESTMENT ANALYSIS")
print("-" * 40)

# Price trend analysis
if len(recent) > 0:
    # Calculate price trends over recent years
    yearly_trends = recent.groupby(['town', 'year']).agg({
        'price_per_sqm': 'mean',
        'resale_price': 'mean'
    }).reset_index()
    
    # Calculate appreciation rates for towns with multiple years of data
    appreciation_data = []
    
    for town in recent['town'].unique():
        town_data = yearly_trends[yearly_trends['town'] == town]
        if len(town_data) >= 2:
            town_data = town_data.sort_values('year')
            years_span = town_data['year'].max() - town_data['year'].min()
            
            if years_span > 0:
                start_price = town_data['price_per_sqm'].iloc[0]
                end_price = town_data['price_per_sqm'].iloc[-1]
                
                # Calculate annualized growth rate
                annual_growth = ((end_price / start_price) ** (1/years_span) - 1) * 100
                
                appreciation_data.append({
                    'town': town,
                    'annual_growth_%': annual_growth,
                    'years_data': years_span,
                    'start_price_psm': start_price,
                    'end_price_psm': end_price,
                    'total_growth_%': ((end_price / start_price) - 1) * 100
                })
    
    if appreciation_data:
        appreciation_df = pd.DataFrame(appreciation_data)
        appreciation_df = appreciation_df.sort_values('annual_growth_%', ascending=False)
        
        print("📈 PRICE APPRECIATION BY TOWN (Annual %):")
        print(appreciation_df[['town', 'annual_growth_%', 'total_growth_%', 'years_data']].head(10))

# ------------------------------------------------------------
# 9. SPECIFIC PROPERTY RECOMMENDATIONS
# ------------------------------------------------------------
print("\n🎯 TOP PROPERTY RECOMMENDATIONS")
print("-" * 40)

# Create comprehensive scoring system
recent = recent.copy()

# Normalize scores for fair comparison
def normalize_score(series, reverse=False):
    """Normalize to 0-100 scale"""
    min_val, max_val = series.min(), series.max()
    if max_val == min_val:
        return pd.Series([50] * len(series), index=series.index)
    
    if reverse:
        return 100 * (max_val - series) / (max_val - min_val)
    else:
        return 100 * (series - min_val) / (max_val - min_val)

# Calculate component scores
recent['size_score'] = normalize_score(recent['floor_area_sqm'])  # Bigger is better
recent['price_efficiency_score'] = normalize_score(recent['price_per_sqm'], reverse=True)  # Lower $/sqm is better
recent['lease_score'] = normalize_score(recent['remaining_lease_years'])  # More lease is better
recent['location_score_normalized'] = (recent['location_score'] / 10) * 100

# Overall value score with weights
weights = {
    'location': 0.30,      # 30% - Location is key
    'price_efficiency': 0.25,  # 25% - Value for money
    'size': 0.25,         # 25% - Size matters
    'lease': 0.20          # 20% - Remaining lease
}

recent['overall_score'] = (
    weights['location'] * recent['location_score_normalized'] +
    weights['price_efficiency'] * recent['price_efficiency_score'] +
    weights['size'] * recent['size_score'] +
    weights['lease'] * recent['lease_score']
).round(2)

# Top recommendations
print("🏆 TOP 15 SPECIFIC PROPERTIES:")
top_properties = recent.nlargest(15, 'overall_score')[
    ['town', 'block', 'street_name', 'resale_price', 'floor_area_sqm', 
     'price_per_sqm', 'remaining_lease_years', 'location_tier', 'overall_score', 'month']
]

for i, (_, prop) in enumerate(top_properties.iterrows(), 1):
    print(f"\n{i:2d}. 🏠 Block {prop['block']} {prop['street_name']}")
    print(f"     📍 {prop['town']} ({prop['location_tier']})")
    print(f"     💰 ${prop['resale_price']:,.0f} | {prop['floor_area_sqm']:.0f} sqm | ${prop['price_per_sqm']:,.0f}/sqm")
    print(f"     ⏰ {prop['remaining_lease_years']:.1f} years lease | Score: {prop['overall_score']:.1f}/100")
    print(f"     📅 {prop['month'].strftime('%b %Y')}")

# ------------------------------------------------------------
# 10. SUMMARY & INVESTMENT INSIGHTS
# ------------------------------------------------------------
print("\n" + "="*60)
print("📊 INVESTMENT SUMMARY & INSIGHTS")
print("="*60)

print(f"\n🎯 MARKET OVERVIEW:")
print(f"   • Properties meeting criteria: {len(recent):,}")
print(f"   • Price range: ${recent['resale_price'].min():,.0f} - ${recent['resale_price'].max():,.0f}")
print(f"   • Average price: ${recent['resale_price'].mean():,.0f}")
print(f"   • Size range: {recent['floor_area_sqm'].min():.0f} - {recent['floor_area_sqm'].max():.0f} sqm")
print(f"   • Average size: {recent['floor_area_sqm'].mean():.1f} sqm")
print(f"   • Available towns: {recent['town'].nunique()}")

print(f"\n🏆 TOP 5 RECOMMENDED TOWNS:")
for i, (town, data) in enumerate(town_stats.head(5).iterrows(), 1):
    print(f"{i}. {town} ({data['location_tier']})")
    print(f"   💰 Avg: ${data['price_mean']:,.0f} | {data['sqm_mean']:.0f} sqm | ${data['psf_mean']:,.0f}/sqm")
    print(f"   📊 {data['count']:.0f} properties | Score: {data['value_score']:.1f}/100")

print(f"\n💡 KEY INSIGHTS:")
print(f"   • Best value towns balance location, size, and price")
print(f"   • Central areas command premium but offer better liquidity")
print(f"   • Some 3-room flats offer more space than 4-room options")
print(f"   • Focus on remaining lease >70 years for future resale value")
print(f"   • Consider transport connectivity for 5-year exit strategy")

# Save results
try:
    recent.to_csv('hdb_investment_analysis.csv', index=False)
    town_stats.to_csv('hdb_town_statistics.csv')
    print(f"\n💾 Analysis saved to 'hdb_investment_analysis.csv' and 'hdb_town_statistics.csv'")
except:
    print(f"\n⚠️ Could not save files - check permissions")

print(f"\n✅ Analysis complete! Use the CSV files for further research.")
print(f"   Consider visiting shortlisted properties and checking transport links manually.")

🏠 HDB 4-ROOM FLAT INVESTMENT ANALYSIS

📊 DATA EXPLORATION
----------------------------------------
Dataset shape: (215002, 14)
Columns: ['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_year', 'remaining_lease', 'resale_price', 'remaining_lease_years', 'year', 'flat_age']

Sample data:
       month        town flat_type block        street_name storey_range  \
0 2017-01-01  ANG MO KIO    2 ROOM   406  ANG MO KIO AVE 10     10 TO 12   
1 2017-01-01  ANG MO KIO    3 ROOM   108   ANG MO KIO AVE 4     01 TO 03   
2 2017-01-01  ANG MO KIO    3 ROOM   602   ANG MO KIO AVE 5     01 TO 03   

   floor_area_sqm      flat_model  lease_commence_year     remaining_lease  \
0            44.0        Improved                 1979  61 years 04 months   
1            67.0  New Generation                 1978  60 years 07 months   
2            67.0  New Generation                 1980  62 years 05 months   

   resale_price  remaining

## 

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set up plotting
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("="*70)
print("🏠 HDB 4-ROOM INVESTMENT ANALYSIS - LAST 18 MONTHS")
print("="*70)
print("📅 Analysis Date: September 2025")
print("🔍 Transaction Period: March 2024 - September 2025")

# ------------------------------------------------------------
# 1. DATA PREPARATION & EXPLORATION
# ------------------------------------------------------------
def explore_data_structure():
    """Explore the dataset structure first"""
    print("\n📊 DATA EXPLORATION")
    print("-" * 50)
    print(f"Dataset shape: {resale.shape}")
    print(f"Columns: {list(resale.columns)}")
    print(f"\nSample data:")
    print(resale.head(2))
    print(f"\nData types:")
    print(resale.dtypes)
    
    # Check date range
    if 'month' in resale.columns:
        resale_temp = resale.copy()
        resale_temp['month'] = pd.to_datetime(resale_temp['month'])
        print(f"\nDate range: {resale_temp['month'].min()} to {resale_temp['month'].max()}")
        print(f"Total months: {resale_temp['month'].nunique()}")

explore_data_structure()

# ------------------------------------------------------------
# 2. PARSE REMAINING LEASE
# ------------------------------------------------------------
def parse_remaining_lease(lease_str):
    """Convert remaining lease string to years (float)"""
    if pd.isna(lease_str):
        return np.nan
    
    lease_str = str(lease_str).strip().lower()
    
    years = 0
    months = 0
    
    # Handle different formats: "95 years 03 months", "95 years", "95"
    if 'year' in lease_str:
        parts = lease_str.split('year')
        try:
            years = int(parts[0].strip().split()[-1])
        except:
            years = 0
            
        # Check for months after years
        if len(parts) > 1 and 'month' in parts[1]:
            try:
                months_part = parts[1].split('month')[0].strip()
                months = int(months_part.split()[-1])
            except:
                months = 0
    else:
        # If no 'years' keyword, assume it's just a number
        try:
            years = int(float(lease_str))
        except:
            years = 0
    
    return years + months/12

# ------------------------------------------------------------
# 3. CLEAN AND PREPARE DATA
# ------------------------------------------------------------
print("\n🧹 DATA CLEANING & PREPARATION")
print("-" * 50)

# Parse dates and remaining lease
resale['month'] = pd.to_datetime(resale['month'])
resale['year'] = resale['month'].dt.year
resale['remaining_lease_years'] = resale['remaining_lease'].apply(parse_remaining_lease)

# Calculate flat age - try different possible column names
age_calculated = False
lease_cols = ['lease_commence_date', 'lease_commencement_date', 'lease_start_date']

for col in lease_cols:
    if col in resale.columns:
        current_year = 2025  # Updated to 2025
        resale['flat_age'] = current_year - resale[col]
        print(f"✓ Calculated flat age using: {col}")
        age_calculated = True
        break

if not age_calculated:
    # Calculate from remaining lease (assuming 99-year lease)
    resale['flat_age'] = 99 - resale['remaining_lease_years']
    print("✓ Calculated flat age from remaining lease (99-year assumption)")

# Calculate price per sqm
resale['price_per_sqm'] = resale['resale_price'] / resale['floor_area_sqm']

print(f"Original records: {len(resale):,}")

# ------------------------------------------------------------
# 4. APPLY YOUR INVESTMENT CRITERIA (Updated for 18 months)
# ------------------------------------------------------------
print("\n🎯 APPLYING INVESTMENT CRITERIA")
print("-" * 50)

# Your specific requirements with 18-month transaction window
criteria = {
    'price_min': 600_000,
    'price_max': 800_000,      # Extended for better picture
    'flat_type': '4 ROOM',
    'remaining_lease_min': 70,
    'max_flat_age': 20,
    'transaction_months': 18   # Last 18 months only
}

print("Investment Criteria:")
for key, value in criteria.items():
    print(f"  • {key}: {value}")

# Calculate 18-month cutoff date (from Sep 2025)
cutoff_date = datetime(2025, 9, 1) - timedelta(days=30.44 * criteria['transaction_months'])
print(f"\n📅 Transaction cutoff date: {cutoff_date.strftime('%B %Y')}")

# Step-by-step filtering
filtered = resale.copy()

# 0. Last 18 months transactions only
filtered = filtered[filtered['month'] >= cutoff_date]
print(f"\n0️⃣ Last 18 months transactions: {len(filtered):,}")

if len(filtered) == 0:
    print("⚠️ No transactions in last 18 months, extending to 24 months")
    cutoff_date = datetime(2025, 9, 1) - timedelta(days=30.44 * 24)
    filtered = resale[resale['month'] >= cutoff_date]
    print(f"📊 Using last 24 months: {len(filtered):,} transactions")

# 1. Price range
filtered = filtered[
    (filtered['resale_price'] >= criteria['price_min']) & 
    (filtered['resale_price'] <= criteria['price_max'])
]
print(f"1️⃣ After price filter (${criteria['price_min']:,}-${criteria['price_max']:,}): {len(filtered):,}")

# 2. 4-room flats only
filtered = filtered[filtered['flat_type'] == criteria['flat_type']]
print(f"2️⃣ After 4-room filter: {len(filtered):,}")

# 3. Remaining lease >= 70 years
filtered = filtered[filtered['remaining_lease_years'] >= criteria['remaining_lease_min']]
print(f"3️⃣ After remaining lease ≥{criteria['remaining_lease_min']} years: {len(filtered):,}")

# 4. Flat age <= 20 years (houses used for max 20 years)
filtered = filtered[filtered['flat_age'] <= criteria['max_flat_age']]
print(f"4️⃣ After flat age ≤{criteria['max_flat_age']} years: {len(filtered):,}")

recent = filtered.copy()

# ------------------------------------------------------------
# 5. LOCATION ANALYSIS WITH MRT/TRANSPORT FOCUS
# ------------------------------------------------------------
print("\n📍 LOCATION & TRANSPORT CONNECTIVITY ANALYSIS")
print("-" * 50)

# Singapore regions by centrality and transport connectivity
location_tiers = {
    'Prime Central + MRT': {
        'towns': ['DOWNTOWN CORE', 'MARINA EAST', 'MARINA SOUTH', 'MUSEUM', 
                 'NEWTON', 'ORCHARD', 'OUTRAM', 'RIVER VALLEY', 'ROCHOR'],
        'score': 10,
        'transport': 'Multiple MRT lines, CBD access',
        'description': 'CBD and prime districts with excellent transport'
    },
    'Central Core + MRT': {
        'towns': ['BUKIT MERAH', 'KALLANG/WHAMPOA', 'QUEENSTOWN', 'TOA PAYOH', 
                 'MARINE PARADE', 'GEYLANG', 'BISHAN'],
        'score': 9,
        'transport': 'Major MRT interchange/lines',
        'description': 'Central areas with excellent MRT connectivity'
    },
    'Transport Hubs': {
        'towns': ['ANG MO KIO', 'CLEMENTI', 'BEDOK', 'TAMPINES', 'JURONG EAST', 
                 'HOUGANG', 'SERANGOON'],
        'score': 8,
        'transport': 'MRT interchange or major hub',
        'description': 'Major transport hubs with multiple lines'
    },
    'Well-Connected MRT': {
        'towns': ['BUKIT BATOK', 'BUKIT PANJANG', 'CHOA CHU KANG', 'PASIR RIS', 
                 'JURONG WEST', 'WOODLANDS', 'YISHUN'],
        'score': 7,
        'transport': 'Single MRT line or LRT',
        'description': 'Good MRT/LRT connectivity'
    },
    'Developing Areas': {
        'towns': ['SEMBAWANG', 'PUNGGOL', 'SENGKANG'],
        'score': 6,
        'transport': 'Newer MRT lines',
        'description': 'Newer areas with developing transport'
    }
}

def get_location_info(town):
    """Get comprehensive location info for a town"""
    town_upper = town.upper()
    for tier_name, tier_info in location_tiers.items():
        if town_upper in [t.upper() for t in tier_info['towns']]:
            return tier_name, tier_info['score'], tier_info['transport'], tier_info['description']
    return 'Other Areas', 5, 'Limited transport options', 'Other areas'

# Apply location scoring
if len(recent) > 0:
    recent = recent.copy()
    location_data = recent['town'].apply(get_location_info)
    recent['location_tier'] = [x[0] for x in location_data]
    recent['location_score'] = [x[1] for x in location_data]
    recent['transport_info'] = [x[2] for x in location_data]
    recent['tier_description'] = [x[3] for x in location_data]

    # Show location distribution
    location_summary = recent.groupby(['location_tier', 'transport_info']).agg({
        'resale_price': ['mean', 'count'],
        'floor_area_sqm': 'mean',
        'price_per_sqm': 'mean',
        'location_score': 'first'
    }).round(0)

    location_summary.columns = ['avg_price', 'count', 'avg_sqm', 'avg_price_per_sqm', 'score']
    location_summary = location_summary.sort_values('score', ascending=False)

    print("📊 Properties by Location & Transport:")
    print(location_summary)

# ------------------------------------------------------------
# 6. 3-ROOM vs 4-ROOM SIZE COMPARISON
# ------------------------------------------------------------
print("\n📏 3-ROOM vs 4-ROOM SIZE COMPARISON (LAST 18 MONTHS)")
print("-" * 50)

# Get 3-room data for comparison (same 18-month period)
three_room = resale[
    (resale['flat_type'] == '3 ROOM') & 
    (resale['month'] >= cutoff_date) &
    (resale['flat_age'] <= 30) &  # Allow older 3-room for size comparison
    (resale['resale_price'] <= 700_000)  # Keep within reasonable price range
].copy()

if len(three_room) > 0 and len(recent) > 0:
    # Size comparison analysis
    size_analysis = pd.DataFrame({
        'Property Type': ['3-Room Flats (≤30 years)', '4-Room Flats (Your Criteria)'],
        'Count (Last 18M)': [len(three_room), len(recent)],
        'Avg Size (sqm)': [three_room['floor_area_sqm'].mean(), recent['floor_area_sqm'].mean()],
        'Median Size (sqm)': [three_room['floor_area_sqm'].median(), recent['floor_area_sqm'].median()],
        'Max Size (sqm)': [three_room['floor_area_sqm'].max(), recent['floor_area_sqm'].max()],
        'Avg Price': [three_room['resale_price'].mean(), recent['resale_price'].mean()],
        'Avg Price/sqm': [three_room['price_per_sqm'].mean(), recent['price_per_sqm'].mean()]
    }).round(2)

    print(size_analysis)

    # Find 3-room flats larger than median 4-room
    if len(recent) > 0:
        median_4room_size = recent['floor_area_sqm'].median()
        large_3room = three_room[three_room['floor_area_sqm'] > median_4room_size]

        print(f"\n📊 3-room flats larger than median 4-room ({median_4room_size:.0f} sqm): {len(large_3room):,}")
        if len(large_3room) > 0:
            print(f"   Average size of large 3-rooms: {large_3room['floor_area_sqm'].mean():.1f} sqm")
            print(f"   Average price: ${large_3room['resale_price'].mean():,.0f}")
            print(f"   Price range: ${large_3room['resale_price'].min():,.0f} - ${large_3room['resale_price'].max():,.0f}")
            
            # Show top large 3-room options
            if len(large_3room) >= 5:
                print(f"\n🏠 TOP 5 LARGE 3-ROOM OPTIONS:")
                large_3room_sorted = large_3room.nlargest(5, 'floor_area_sqm')[
                    ['town', 'block', 'street_name', 'resale_price', 'floor_area_sqm', 'price_per_sqm', 'month']
                ]
                for i, (_, prop) in enumerate(large_3room_sorted.iterrows(), 1):
                    print(f"{i}. {prop['town']} - Block {prop['block']} {prop['street_name']}")
                    print(f"   ${prop['resale_price']:,.0f} | {prop['floor_area_sqm']:.0f} sqm | ${prop['price_per_sqm']:,.0f}/sqm | {prop['month'].strftime('%b %Y')}")

# ------------------------------------------------------------
# 7. COMPREHENSIVE SCORING AND RECOMMENDATIONS
# ------------------------------------------------------------
if len(recent) > 0:
    print("\n🎯 COMPREHENSIVE PROPERTY SCORING")
    print("-" * 50)

    # Create comprehensive scoring system
    recent = recent.copy()

    # Normalize scores for fair comparison
    def normalize_score(series, reverse=False):
        """Normalize to 0-100 scale"""
        if len(series) == 0:
            return series
        min_val, max_val = series.min(), series.max()
        if max_val == min_val:
            return pd.Series([50] * len(series), index=series.index)
        
        if reverse:
            return 100 * (max_val - series) / (max_val - min_val)
        else:
            return 100 * (series - min_val) / (max_val - min_val)

    # Calculate component scores
    recent['size_score'] = normalize_score(recent['floor_area_sqm'])  # Bigger is better
    recent['price_efficiency_score'] = normalize_score(recent['price_per_sqm'], reverse=True)  # Lower $/sqm is better
    recent['lease_score'] = normalize_score(recent['remaining_lease_years'])  # More lease is better
    recent['location_score_normalized'] = (recent['location_score'] / 10) * 100

    # Overall value score with weights (prioritizing location and transport for 5-year exit)
    weights = {
        'location': 0.35,      # 35% - Location critical for resale
        'price_efficiency': 0.25,  # 25% - Value for money
        'size': 0.25,         # 25% - Size matters
        'lease': 0.15          # 15% - Remaining lease (less critical with 70+ requirement)
    }

    recent['overall_score'] = (
        weights['location'] * recent['location_score_normalized'] +
        weights['price_efficiency'] * recent['price_efficiency_score'] +
        weights['size'] * recent['size_score'] +
        weights['lease'] * recent['lease_score']
    ).round(2)

    # ------------------------------------------------------------
    # 8. TOP RECOMMENDATIONS WITH FULL ADDRESSES
    # ------------------------------------------------------------
    print("\n🏆 TOP 20 RECOMMENDED PROPERTIES WITH ADDRESSES")
    print("-" * 50)

    top_properties = recent.nlargest(20, 'overall_score').copy()
    
    # Create full addresses
    top_properties['full_address'] = (
        "Block " + top_properties['block'].astype(str) + " " + 
        top_properties['street_name'].astype(str) + ", Singapore"
    )

    print("🏠 DETAILED PROPERTY RECOMMENDATIONS:")
    print("="*80)

    for i, (_, prop) in enumerate(top_properties.iterrows(), 1):
        print(f"\n{i:2d}. 📍 {prop['full_address']}")
        print(f"     🏘️  {prop['town']} ({prop['location_tier']})")
        print(f"     💰 Price: ${prop['resale_price']:,.0f}")
        print(f"     📐 Size: {prop['floor_area_sqm']:.0f} sqm (${prop['price_per_sqm']:,.0f}/sqm)")
        print(f"     ⏰ Remaining Lease: {prop['remaining_lease_years']:.1f} years")
        print(f"     🚇 Transport: {prop['transport_info']}")
        print(f"     📊 Overall Score: {prop['overall_score']:.1f}/100")
        print(f"     📅 Transaction: {prop['month'].strftime('%B %Y')}")
        print(f"     " + "-"*60)

    # ------------------------------------------------------------
    # 9. TOWN-LEVEL ANALYSIS WITH ADDRESSES
    # ------------------------------------------------------------
    print(f"\n🏘️ TOP RECOMMENDED TOWNS WITH SAMPLE ADDRESSES")
    print("-" * 50)

    town_stats = recent.groupby('town').agg({
        'resale_price': ['mean', 'median', 'min', 'max', 'count'],
        'floor_area_sqm': ['mean', 'median'],
        'price_per_sqm': ['mean', 'median'],
        'remaining_lease_years': ['mean', 'min'],
        'flat_age': 'mean',
        'location_score': 'first',
        'location_tier': 'first',
        'transport_info': 'first',
        'overall_score': 'mean'
    }).round(2)

    # Flatten columns
    town_stats.columns = ['price_mean', 'price_median', 'price_min', 'price_max', 'count',
                         'sqm_mean', 'sqm_median', 'psf_mean', 'psf_median', 
                         'lease_mean', 'lease_min', 'age_mean', 'location_score', 
                         'location_tier', 'transport_info', 'avg_overall_score']

    # Sort by average overall score
    town_stats = town_stats.sort_values('avg_overall_score', ascending=False)

    print("🏆 TOP 10 RECOMMENDED TOWNS:")
    print("="*80)

    for i, (town, data) in enumerate(town_stats.head(10).iterrows(), 1):
        print(f"\n{i:2d}. 🏘️ {town}")
        print(f"     Location Tier: {data['location_tier']}")
        print(f"     🚇 Transport: {data['transport_info']}")
        print(f"     💰 Avg Price: ${data['price_mean']:,.0f} (Range: ${data['price_min']:,.0f}-${data['price_max']:,.0f})")
        print(f"     📐 Avg Size: {data['sqm_mean']:.0f} sqm | ${data['psf_mean']:,.0f}/sqm")
        print(f"     📊 Available Properties: {data['count']:.0f} | Town Score: {data['avg_overall_score']:.1f}/100")
        
        # Show sample addresses for this town
        town_properties = recent[recent['town'] == town].nlargest(3, 'overall_score')
        if len(town_properties) > 0:
            print(f"     🏠 Sample Properties:")
            for j, (_, sample) in enumerate(town_properties.iterrows(), 1):
                print(f"        {j}. Block {sample['block']} {sample['street_name']}")
                print(f"           ${sample['resale_price']:,.0f} | {sample['floor_area_sqm']:.0f} sqm | Score: {sample['overall_score']:.1f}")
        print(f"     " + "-"*70)

    # ------------------------------------------------------------
    # 10. INVESTMENT SUMMARY
    # ------------------------------------------------------------
    print(f"\n" + "="*80)
    print("📊 INVESTMENT SUMMARY & NEXT STEPS")
    print("="*80)

    print(f"\n🎯 MARKET OVERVIEW (Last 18 Months):")
    print(f"   • Total properties meeting all criteria: {len(recent):,}")
    print(f"   • Price range: ${recent['resale_price'].min():,.0f} - ${recent['resale_price'].max():,.0f}")
    print(f"   • Average price: ${recent['resale_price'].mean():,.0f}")
    print(f"   • Size range: {recent['floor_area_sqm'].min():.0f} - {recent['floor_area_sqm'].max():.0f} sqm")
    print(f"   • Average size: {recent['floor_area_sqm'].mean():.1f} sqm")
    print(f"   • Available in {recent['town'].nunique()} towns")

    print(f"\n💡 KEY INSIGHTS:")
    print(f"   • Focus on top-scoring properties in well-connected areas")
    print(f"   • Central/transport hub locations offer better 5-year exit liquidity")
    print(f"   • Some larger 3-room flats may offer better value than smaller 4-room")
    print(f"   • All recommendations meet your 70+ year lease requirement")
    print(f"   • Properties are from verified transactions in last 18 months")

    print(f"\n🎯 NEXT STEPS:")
    print(f"   1. Visit top 5-10 properties from the detailed list above")
    print(f"   2. Check actual MRT distances and travel times to key locations")
    print(f"   3. Verify current market conditions with property agents")
    print(f"   4. Consider upcoming BTO launches that might affect resale values")
    print(f"   5. Factor in renovation costs for 5-year investment timeline")

    # Save results with addresses
    try:
        # Save top properties with full details
        top_properties_export = top_properties[[
            'full_address', 'town', 'block', 'street_name', 'resale_price', 
            'floor_area_sqm', 'price_per_sqm', 'remaining_lease_years', 
            'flat_age', 'location_tier', 'transport_info', 'overall_score', 'month'
        ]].copy()
        
        top_properties_export.to_csv('hdb_top_properties_with_addresses.csv', index=False)
        
        # Save town analysis
        town_stats.to_csv('hdb_town_analysis_18months.csv')
        
        # Save all filtered data
        recent.to_csv('hdb_filtered_data_18months.csv', index=False)
        
        print(f"\n💾 Analysis saved to:")
        print(f"   • 'hdb_top_properties_with_addresses.csv' - Top properties with full addresses")
        print(f"   • 'hdb_town_analysis_18months.csv' - Town-level statistics")
        print(f"   • 'hdb_filtered_data_18months.csv' - All filtered properties")
        
    except Exception as e:
        print(f"\n⚠️ Could not save files: {e}")

else:
    print(f"\n⚠️ No properties found meeting your criteria in the last 18 months.")
    print(f"Consider:")
    print(f"   • Extending the time period to 24-30 months")
    print(f"   • Adjusting the price range slightly")
    print(f"   • Including flats with 65+ years remaining lease")
    print(f"   • Looking at 3-room alternatives that may be larger")

print(f"\n✅ Analysis complete!")

🏠 HDB 4-ROOM INVESTMENT ANALYSIS - LAST 18 MONTHS
📅 Analysis Date: September 2025
🔍 Transaction Period: March 2024 - September 2025

📊 DATA EXPLORATION
--------------------------------------------------
Dataset shape: (215002, 15)
Columns: ['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_year', 'remaining_lease', 'resale_price', 'remaining_lease_years', 'year', 'flat_age', 'price_per_sqm']

Sample data:
       month        town flat_type block        street_name storey_range  \
0 2017-01-01  ANG MO KIO    2 ROOM   406  ANG MO KIO AVE 10     10 TO 12   
1 2017-01-01  ANG MO KIO    3 ROOM   108   ANG MO KIO AVE 4     01 TO 03   

   floor_area_sqm      flat_model  lease_commence_year     remaining_lease  \
0            44.0        Improved                 1979  61 years 04 months   
1            67.0  New Generation                 1978  60 years 07 months   

   resale_price  remaining_lease_years  year   flat_age  pr